# Azure Open AI - Demo 4 Speech summarizing

In [1]:
import azure.cognitiveservices.speech as speechsdk
import openai
import os
import sys
import time

from dotenv import load_dotenv
from IPython.display import Audio

In [2]:
sys.version

'3.10.10 (main, Mar 21 2023, 18:45:11) [GCC 11.2.0]'

In [3]:
print("openai version =", openai.__version__)

openai version = 0.28.0


In [4]:
load_dotenv("azure.env")

openai.api_type: str = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

azure_speech_key = os.getenv("AZURE_SPEECH_KEY")
azure_speech_region = os.getenv("AZURE_SPEECH_REGION")

## 1. Convert Audio To Text Using Azure Speech To Text  

In [5]:
def recognize_speech_from_file(filename):
    """This function transcribes an audio file into its corresponding textual output."""

    # setup global parameters
    global done
    done = False
    global recognized_text_list
    recognized_text_list = []

    # initiate the Speech Service
    speech_config = speechsdk.SpeechConfig(
        subscription=azure_speech_key, region=azure_speech_region
    )
    audio_config = speechsdk.audio.AudioConfig(filename=filename)

    # create a Speech Recognizer
    speech_recognizer = speechsdk.SpeechRecognizer(
        speech_config=speech_config, audio_config=audio_config
    )

    # define a call-back stop function
    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print("CLOSING on {}".format(evt))
        global done
        done = True

    # define a call-back recognize function
    def recognize_cb(evt: speechsdk.SpeechRecognitionEventArgs):
        """callback for recognizing the recognized text"""
        global recognized_text_list
        recognized_text_list.append(evt.result.text)
        print("RECOGNIZED: {}".format(evt.result.text))

    # connect callbacks to the recognizer events
    speech_recognizer.recognized.connect(recognize_cb)
    speech_recognizer.session_started.connect(
        lambda evt: print("STT SESSION STARTED: {}".format(evt))
    )
    speech_recognizer.session_stopped.connect(
        lambda evt: print("STT SESSION STOPPED {}".format(evt))
    )

    # stop continuous speech recognition
    speech_recognizer.session_stopped.connect(stop_cb)

    # start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(0.5)

    speech_recognizer.stop_continuous_recognition()

    return recognized_text_list

## 2. Summarize Audio - Positive Example 

In [6]:
!ls documents/good_review.zip -lh

-rwxrwxrwx 1 root root 967K Sep  8 12:00 documents/good_review.zip


In [7]:
os.chdir("documents")
!unzip good_review.zip
os.chdir("..")

Archive:  good_review.zip
  inflating: good_review.wav         


In [8]:
!ls documents/good_review.wav -lh

-rwxrwxrwx 1 root root 1.4M Mar  4  2023 documents/good_review.wav


In [9]:
# play the audio sound
Audio("documents/good_review.wav", autoplay=False)

In [10]:
text = recognize_speech_from_file("documents/good_review.wav")

STT SESSION STARTED: SessionEventArgs(session_id=34b924393ad2449b91ee644c1c001f8c)
RECOGNIZED: I had purchased another thermostat from a big box store and the Rep there assured me that it would turn off when it got too cold.
RECOGNIZED: Well, come to find out, it did not and my heater in the garage was running a lot of the time.
RECOGNIZED: This one was great and was easily wired.
RECOGNIZED: It does turn off when you turn it all the way to the left.
RECOGNIZED: It's annoying they don't have any in Celsius, but whatever.
RECOGNIZED: It runs well and actually turns off my garage heater.
RECOGNIZED: Would totally purchase again.
RECOGNIZED: Installed with five kilowatts heater in garage.
RECOGNIZED: Thermostat failed an on position overnight and came close to fire with overheated connections. See photos.
RECOGNIZED: Installation was by certified electrician.
STT SESSION STOPPED SessionEventArgs(session_id=34b924393ad2449b91ee644c1c001f8c)
CLOSING on SessionEventArgs(session_id=34b924393a

In [11]:
model = "text-davinci-003"

In [12]:
# create a prompt for text analytics
prompt = f"""Text:

             ###
             {" ".join(text)}
             ###
             
             Tasks:
             1. Summarize the above text in a short and concise manner.
             2. Determine the above text as whether Positive or Negative.
             3. Classify the above text into a topic.
             4. Whether the above text is Positive or Negative, 
                suggest 3 ways to handle it.
                
             Answer the above 4 tasks, one at a time. 
          """

results = openai.Completion.create(
    engine=model, prompt=prompt, temperature=0, max_tokens=1000
)

print(results["choices"][0]["text"])


1. The text is about a customer's experience with a thermostat purchased from a big box store. The customer was initially disappointed as the thermostat did not turn off when it got too cold, but was pleased when they found a thermostat that did turn off and ran well.

2. The text is Positive.

3. The topic of the text is customer experience.

4. If the text is Positive:
   a. Acknowledge the customer's positive experience and thank them for their feedback.
   b. Ask the customer if they would be willing to leave a review on the product page.
   c. Offer the customer a discount or coupon code for their next purchase.


In [13]:
prompt = f"""Text:

             ###
             {" ".join(text)}
             ###
             
             Tasks:
             1. Summarize the above text in a short and concise manner.
             2. Determine the above text as whether Positive or Negative.
             3. Classify the above text into a topic.
             4. Whether the above text is Positive or Negative, 
                suggest 3 ways to handle it.
                
             Answer the above 4 tasks, one at a time, in French. 
          """

# openai results
results = openai.Completion.create(
    engine=model, prompt=prompt, temperature=0, max_tokens=1000
)

print(results["choices"][0]["text"])


1. Le texte décrit une expérience d'achat et d'installation d'un thermostat qui s'est avérée satisfaisante, car il s'éteint lorsque le réglage est à gauche.
2. Le texte est positif.
3. Le texte appartient au domaine de la technologie et des appareils électroniques.
4. Si le texte est positif, les 3 façons de le gérer sont : offrir des conseils et des informations supplémentaires aux clients, améliorer la qualité des produits et des services et offrir des remises et des promotions pour encourager les clients à acheter.


In [14]:
prompt = f"""Text:

             ###
             {" ".join(text)}
             ###
             
             Tasks:
             1. Summarize the above text in a short and concise manner.
             2. Determine the above text as whether Positive or Negative.
             3. Classify the above text into a topic.
             4. Whether the above text is Positive or Negative, 
                suggest 3 ways to handle it.
                
             Answer the above 4 tasks, one at a time, in Spanish. 
          """

# openai results
results = openai.Completion.create(
    engine=model, prompt=prompt, temperature=0, max_tokens=1000
)

print(results["choices"][0]["text"])


1. El texto describe la experiencia de una persona al comprar un termostato de una tienda de grandes almacenes, que no cumplió con sus expectativas, pero que finalmente funcionó bien.
2. El texto es positivo.
3. El tema del texto es la experiencia de compra de un termostato.
4. Si el texto es positivo:
    i. Ofrecer un descuento en la próxima compra.
    ii. Ofrecer una garantía extendida.
    iii. Ofrecer una solución de reemplazo.


## 3. Summarize Audio - Negative Example 

In [15]:
!ls documents/bad_review.zip -lh

-rwxrwxrwx 1 root root 318K Sep  8 12:00 documents/bad_review.zip


In [16]:
os.chdir("documents")
!unzip bad_review.zip
os.chdir("..")

Archive:  bad_review.zip
  inflating: bad_review.wav          


In [17]:
!ls documents/bad_review.wav -lh

-rwxrwxrwx 1 root root 427K Mar  4  2023 documents/bad_review.wav


In [18]:
Audio("documents/bad_review.wav", autoplay=False)

In [19]:
text = recognize_speech_from_file("documents/bad_review.wav")

STT SESSION STARTED: SessionEventArgs(session_id=c7b30764de6f463489926f18077d7b0a)
RECOGNIZED: Installed with five kilowatts heater in garage.
RECOGNIZED: Thermostat failed in on position overnight and came close to fire with overheated connections. See photos.
RECOGNIZED: Installation was by certified electrician.
STT SESSION STOPPED SessionEventArgs(session_id=c7b30764de6f463489926f18077d7b0a)
CLOSING on SessionEventArgs(session_id=c7b30764de6f463489926f18077d7b0a)


In [22]:
prompt = f"""Text:

             ###
             {" ".join(text)}
             ###
             
             Tasks:
             1. Summarize the above text in a short and concise manner.
             2. Determine the above text as whether Positive or Negative.
             3. Classify the above text into a topic.
             4. Whether the above text is Positive or Negative, 
                suggest 3 ways to handle it.
                
             Answer the above 4 tasks, one at a time. 
          """

# openai results
results = openai.Completion.create(
    engine=model, prompt=prompt, temperature=0, max_tokens=1000
)

print(results["choices"][0]["text"])


1. An electrician installed a five kilowatt heater in a garage, but the thermostat failed and caused the connections to overheat, putting the garage at risk of a fire.
2. Negative
3. Home Safety
4. If the text is Negative:
    a. Replace the thermostat with a new one.
    b. Have the electrician inspect the connections for any damage.
    c. Install a fire alarm in the garage.


In [23]:
prompt = f"""Text:

             ###
             {" ".join(text)}
             ###
             
             Tasks:
             1. Summarize the above text in a short and concise manner.
             2. Determine the above text as whether Positive or Negative.
             3. Classify the above text into a topic.
             4. Whether the above text is Positive or Negative, 
                suggest 3 ways to handle it.
                
             Answer the above 4 tasks, one at a time, in French.
          """
# openai results
results = openai.Completion.create(
    engine=model, prompt=prompt, temperature=0, max_tokens=1000
)

print(results["choices"][0]["text"])


1. Une installation d'un chauffage de cinq kilowatts dans le garage a échoué, ce qui a presque causé un incendie à cause des connexions surchauffées.
2. Négatif
3. Sécurité
4. Négatif:
    a. Débrancher le chauffage immédiatement.
    b. Vérifier les connexions et les remplacer si nécessaire.
    c. Faire inspecter l'installation par un électricien certifié.


In [24]:
prompt = f"""Text:

             ###
             {" ".join(text)}
             ###
             
             Tasks:
             1. Summarize the above text in a short and concise manner.
             2. Determine the above text as whether Positive or Negative.
             3. Classify the above text into a topic.
             4. Whether the above text is Positive or Negative, 
                suggest 3 ways to handle it.
                
             Answer the above 4 tasks, one at a time, in Italian.
          """
# openai results
results = openai.Completion.create(
    engine=model, prompt=prompt, temperature=0, max_tokens=1000
)

print(results["choices"][0]["text"])


1. Un riscaldatore da cinque kilowatt è stato installato nel garage. Il termostato è fallito e ha portato vicino al rischio di incendio a causa di connessioni surriscaldate.
2. Negativo
3. Sicurezza
4. Negativo: 
    i. Sostituire il termostato.
    ii. Controllare le connessioni per assicurarsi che non siano surriscaldate.
    iii. Assicurarsi che l'installazione sia stata effettuata da un elettricista certificato.
